In [ ]:
###############################################
# Marche classique vs quantique 4 sites (Qiskit 1.0 compatible)
###############################################

from qiskit import QuantumCircuit
from qiskit.circuit.library import UnitaryGate
from qiskit_aer import AerSimulator
import numpy as np
import matplotlib.pyplot as plt

# ------------------------------------------------
# 1. Marche CLASSIQUE
# ------------------------------------------------

def classical_walk_4sites(n_steps, n_sites=4, start_pos=2):
    P_t = np.zeros(n_sites)
    P_t[start_pos] = 1.0
    distributions = [P_t.copy()]

    for _ in range(n_steps):
        P_next = np.zeros_like(P_t)
        for x in range(n_sites):
            P_next[x] = 0.5 * P_t[(x - 1) % n_sites] + 0.5 * P_t[(x + 1) % n_sites]
        P_t = P_next
        distributions.append(P_t.copy())

    return distributions


# ------------------------------------------------
# 2. Shift quantique
# ------------------------------------------------

def shift_gate_4sites():
    S = np.zeros((8, 8), dtype=complex)
    for coin in [0, 1]:
        for pos in range(4):
            if coin == 0:
                pos_new = (pos - 1) % 4
            else:
                pos_new = (pos + 1) % 4
            in_index = 4 * coin + pos
            out_index = 4 * coin + pos_new
            S[out_index, in_index] = 1.0
    return UnitaryGate(S, label="Shift")

shift = shift_gate_4sites()


# ------------------------------------------------
# 3. Step quantique
# ------------------------------------------------

def one_quantum_step(qc):
    qc.h(2)
    qc.append(shift, [0, 1, 2])


# ------------------------------------------------
# 4. Circuit marche quantique
# ------------------------------------------------

def quantum_walk_4sites_circuit(n_steps, start_pos=2):
    qc = QuantumCircuit(3, 2)
    if start_pos & 0b01: qc.x(0)
    if start_pos & 0b10: qc.x(1)

    for _ in range(n_steps):
        one_quantum_step(qc)

    qc.measure(0, 0)
    qc.measure(1, 1)
    return qc


# ------------------------------------------------
# 5. Distribution quantique (compatible Qiskit 1.0)
# ------------------------------------------------

def quantum_walk_4sites_distributions(n_steps, shots=4096, start_pos=2):
    backend = AerSimulator()
    distributions = []

    P0 = np.zeros(4)
    P0[start_pos] = 1.0
    distributions.append(P0)

    for steps in range(1, n_steps + 1):
        qc = quantum_walk_4sites_circuit(steps, start_pos)
        job = backend.run(qc, shots=shots)
        result = job.result()
        counts = result.get_counts()
        P = np.zeros(4)
        for bitstr, c in counts.items():
            pos = int(bitstr, 2)
            P[pos] += c / shots
        distributions.append(P)

    return distributions


# ------------------------------------------------
# 6. Plot CLASSIQUE vs QUANTIQUE
# ------------------------------------------------

if __name__ == "__main__":
    n_steps = 6
    start_pos = 2

    classical_dists = classical_walk_4sites(n_steps, start_pos=start_pos)
    quantum_dists   = quantum_walk_4sites_distributions(n_steps, start_pos=start_pos)

    fig, axes = plt.subplots(2, n_steps + 1, figsize=(3*(n_steps+1), 6), sharey=True)

    for t in range(n_steps + 1):
        # CLASSIQUE
        axes[0, t].bar(range(4), classical_dists[t])
        axes[0, t].set_title(f"Classique, t={t}")
        axes[0, t].set_xticks(range(4))
        if t == 0:
            axes[0, t].set_ylabel("Probabilité")

        # QUANTIQUE
        axes[1, t].bar(range(4), quantum_dists[t])
        axes[1, t].set_title(f"Quantique, t={t}")
        axes[1, t].set_xticks(range(4))
        if t == 0:
            axes[1, t].set_ylabel("Probabilité")

    plt.tight_layout()
    plt.show()
